In [1]:
import json

In [2]:
import time

In [3]:
from tqdm import tqdm

In [4]:
import os

In [5]:
import re
import ast

In [5]:
import amrlib
import spacy
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')

/data/wang/zhenyub/condaenvs/cenv_x86/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [6]:
from openai import OpenAI
client = OpenAI(api_key='YOUR_KEY')

# Set up YOUR OpenAI API key
#OPENAI_API_KEY = "YOUR_KEY"
# Set MODEL to "gpt-3.5-turbo"
model = "gpt-3.5-turbo"
# Set temperature to 0.9, it can be from 0~1, 0 is the most conservative, 1 is the most creative
temperature = 0.5
# messages = [
#     {"role": "system", "content": "You are a helpful and honest assistant. \
#       Respond concisely and truthfully."},
#     {"role": "user", "content": "Explain Einstein's relativity theory in three sentences."},
# ]

# # Send the message to the bot
# response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature)

# # Get the response from the bot
# response = response_message.choices[0].message.content

# print(response)

In [7]:
def Answer_Agent_IO(question='', num_seq=1):
    message = \
    f"""Given a question, give out only the direct answer, without any explanations. \
    Here are two examples as guideline: \
    "Question: John bought 3 cars, each worth $30000. How much did John pay in total for his 3 cars? \
    Answer: ###$9000###. \
    Question: Payton has 9 apples that she is planning to evenly split to 3 of her friends. How many apples would each of her friend get? \
    Answer: ###3###." \
    Give out the final answer in numerical format. \
    Make sure to include your answer in the format: ###answer###. \
    Question: {question} \
    Answer: \
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature, n=num_seq)

    response = [response_message.choices[i].message.content for i in range(num_seq)]

    return response

In [8]:
def Answer_Agent_CoT_2Shot(question='', num_seq=1):
    message = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Here are two examples as guideline: \
    "Question: John bought 3 cars, each worth $30000. He also bought 3 bikes, each worth $1000. How much did John pay in total? \
    Thought: John paid 3 * $30000 = $90000 for the cars, and 3 *$ 1000 = $3000 for the bikes. The total amounts to $90000 + $3000 = $93000. \
    Answer: ###$93000###. \
    Question: Each day, Payton brings 9 apples that she would split to 3 of her friends. In 3 days, how many apples would each of her friend get in total? \
    Thought: Each of the 3 friends would get 9 / 3 = 3 apples per day. For 3 days, each of her friend would get 3 * 3 = 9 apples in total. \
    Answer: ###9###." \
    Give out the answer in numerical format. \
    Make sure to include your final answer in the format: ###answer###. \
    Question: {question} \
    Answer: \
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature, n=num_seq)

    response = [response_message.choices[i].message.content for i in range(num_seq)]
    # response = response_message.choices[0].message.content

    return response

In [9]:
def Answer_Agent_CoT_0Shot(question='', num_seq=1):
    message = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your final answer to the question in the format: ###your final answer###. \
    Give out the answer in numerical format. \
    Question: {question} \
    Let's think step by step. \
    Answer: \
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    #response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature)

    response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature, n=num_seq)

    response = [response_message.choices[i].message.content for i in range(num_seq)]
    # response = response_message.choices[0].message.content

    return response

In [22]:
s = '###123###'
ast.literal_eval(re.search('###(.*)###', s).group(1))

123

In [31]:
s = """Step 1: Find the total number of outcomes when rolling a fair 6-sided die 5 times. Since each roll has 6 possible outcomes, the total number of outcomes is \\(6^5\\).\n\nStep 2: Find the number of ways to get a 6 in at most 2 of the rolls. This can happen in the following ways:\n- 0 sixes: This can happen in \\({5 \\choose 0}\\) ways.\n- 1 six: This can happen in \\({5 \\choose 1}\\) ways.\n- 2 sixes: This can happen in \\({5 \\choose 2}\\) ways.\n\nStep 3: Calculate the total number of favorable outcomes by summing the number of ways to get 0, 1, or 2 sixes.\n\nStep 4: Calculate the probability by dividing the total number of favorable outcomes by the total number of outcomes.\n\n###Your final answer: 0.8796###"""
re.search('###(.*)###', s).group(1)

'Your final answer: 0.8796'

In [10]:
def testing(prompt='', num_agent=1, rounds=1):
    all_answer_gpt = []
    ori_answers_io = []
    fnames_list = []
    rootdir = "test"
    for subdir, dirs, files in os.walk(rootdir):
        print('current directory:', subdir)
        file_num = 0
        for file in files:
            file_num += 1
            if file_num > 14:
                break
            fnames_list.append(os.path.join(subdir, file))
            with open(os.path.join(subdir, file), 'r') as fp:
                try:
                    problem_data = json.load(fp)
                except Exception as e:
                    print(f"Error loading JSON from {file}", e)
                    raise e
                prob_level = problem_data["level"]
                #prob_type = problem_data["type"]
                try:
                    prob_level = int(prob_level.split("Level ")[1])
                except:
                    prob_level = None
                if prompt == 'IO':
                    response = Answer_Agent_IO(problem_data['problem'], 1)[0]
                elif prompt == 'CoT_2Shot':
                    response = Answer_Agent_CoT_2Shot(problem_data['problem'], 1)[0]
                elif prompt == 'CoT_0Shot':
                    response = Answer_Agent_CoT_0Shot(problem_data['problem'], 1)[0]
                # elif prompt == 'CoT_0Shot_Agent':
                #     response = Answer_Agent_debate(problem_data['problem'], rounds, num_agent)
                ori_answers_io.append(response)
                try:
                    response_format = re.search('###(.*)###', response).group(1)
                    try:
                        answer_current = ast.literal_eval(response_format)
                    except:
                        answer_current = response_format
                    all_answer_gpt.append(answer_current)
                except:
                    try:
                        answer_current = ast.literal_eval(response)
                        all_answer_gpt.append(answer_current)
                    except:
                        try:
                            response_format = response.split('Answer: ')[1]
                            answer_current = ast.literal_eval(response_format)
                            all_answer_gpt.append(answer_current)
                        except:
                            print('no answer provided')
                            print(response)
                            all_answer_gpt.append('')
                            continue
    return all_answer_gpt, ori_answers_io

In [17]:
gpt_io, ori_io = testing('IO')

current directory: test
current directory: test/counting_and_probability
current directory: test/intermediate_algebra
current directory: test/number_theory
current directory: test/precalculus
current directory: test/prealgebra
current directory: test/geometry
current directory: test/algebra


In [35]:
gpt_cot_0s, ori_cot_0s = testing('CoT_0Shot')

current directory: test
current directory: test/counting_and_probability
current directory: test/intermediate_algebra
no answer provided
First, let's consider the formula for the sum of cubes of a sequence, which is $\sum_{i=1}^{n} a_i^3 = (a_1 + a_2 + \cdots + a_n)^3 - 3(a_1a_2 + a_1a_3 + \cdots + a_{n-1}a_n)$.

From the given information, we have $\sum_{i=1}^{n} a_i = 96$, $\sum_{i=1}^{n} a_i^2 = 144$, and $\sum_{i=1}^{n} a_i^3 = 216$. Substituting these values into the formula, we get $216 = 96^3 - 3(a_1a_2 + a_1a_3 + \cdots + a_{n-1}a_n)$.

Solving this equation would give us the values of $a_1a_2 + a_1a_3 + \cdots + a_{n-1}a_n$, which are the pairwise products of the elements in the sequence.

To find the sum of all possible values of $n$, we need to determine the factors of 216 that can be expressed as the sum of three positive integers such that their product is the pairwise product of the elements in the sequence.

After calculating the factors of 216 and checking the condition

In [11]:
import sys
import re
import string
from collections import Counter
import pickle

In [12]:
def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [38]:
with open('answers_gpt3_io.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in gpt_io))

In [39]:
with open('answers_gpt3_io_ori.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_io))

In [40]:
with open('answers_gpt3_cot_0Shot_ori.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_cot_0s))

In [41]:
with open('answers_gpt3_cot_0Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in gpt_cot_0s))

In [75]:
with open('answers_gpt3_cot_2Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot))

In [43]:
with open('answers_golden.txt', 'w') as f:
    f.write('\n'.join(s['answer'].split('#### ')[1] for s in split[0:100]))

In [37]:
def remove_boxed(s):
    left = "\\boxed{"
    try:
        assert s[:len(left)] == left
        assert s[-1] == "}"
        return s[len(left):-1]
    except:
        return None

In [13]:
def last_boxed_only_string(string):
    idx = string.rfind("\\boxed")
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return None

    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1
    
    if right_brace_idx == None:
        retval = None
    else:
        retval = string[idx:right_brace_idx + 1]
    
    return retval


In [14]:
ANS_RE = re.compile(r"(\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"

def extract_answer(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        if len(match_str) == 0:
            return INVALID_ANS
        if match_str[-1] == '.':
            match_str = match_str[:-1]
        if len(match_str) == 0:
            return INVALID_ANS
        return match_str
    else:
        return INVALID_ANS
def is_correct(model_completion, gt_example):
    gt_answer = extract_answer(gt_example)
    assert gt_answer != INVALID_ANS
    model_answer = extract_answer(model_completion)
    if model_answer == INVALID_ANS:
        return False
    return int(float(model_answer)) == int(float(gt_answer))

In [35]:
def _fix_fracs(string):
    substrs = string.split("\\frac")
    new_str = substrs[0]
    if len(substrs) > 1:
        substrs = substrs[1:]
        for substr in substrs:
            new_str += "\\frac"
            if substr[0] == "{":
                new_str += substr
            else:
                try:
                    assert len(substr) >= 2
                except:
                    return string
                a = substr[0]
                b = substr[1]
                if b != "{":
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}{" + b + "}" + post_substr
                    else:
                        new_str += "{" + a + "}{" + b + "}"
                else:
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}" + b + post_substr
                    else:
                        new_str += "{" + a + "}" + b
    string = new_str
    return string

def _fix_a_slash_b(string):
    if len(string.split("/")) != 2:
        return string
    a = string.split("/")[0]
    b = string.split("/")[1]
    try:
        a = int(a)
        b = int(b)
        assert string == "{}/{}".format(a, b)
        new_string = "\\frac{" + str(a) + "}{" + str(b) + "}"
        return new_string
    except:
        return string

def _remove_right_units(string):
    # "\\text{ " only ever occurs (at least in the val set) when describing units
    if "\\text{ " in string:
        splits = string.split("\\text{ ")
        assert len(splits) == 2
        return splits[0]
    else:
        return string

def _fix_sqrt(string):
    if "\\sqrt" not in string:
        return string
    splits = string.split("\\sqrt")
    new_string = splits[0] 
    for split in splits[1:]:
        if split[0] != "{":
            a = split[0]
            new_substr = "\\sqrt{" + a + "}" + split[1:]
        else:
            new_substr = "\\sqrt" + split
        new_string += new_substr
    return new_string

def _strip_string(string):
    # linebreaks  
    string = string.replace("\n", "")
    #print(string)

    # remove inverse spaces
    string = string.replace("\\!", "")
    #print(string)

    # replace \\ with \
    string = string.replace("\\\\", "\\")
    #print(string)

    # replace tfrac and dfrac with frac
    string = string.replace("tfrac", "frac")
    string = string.replace("dfrac", "frac")
    #print(string)

    # remove \left and \right
    string = string.replace("\\left", "")
    string = string.replace("\\right", "")
    #print(string)
    
    # Remove circ (degrees)
    string = string.replace("^{\\circ}", "")
    string = string.replace("^\\circ", "")

    # remove dollar signs
    string = string.replace("\\$", "")
    string = string.replace("$", "")
    
    # remove units (on the right)
    string = _remove_right_units(string)

    # remove percentage
    string = string.replace("\\%", "")
    string = string.replace("\%", "")

    # " 0." equivalent to " ." and "{0." equivalent to "{." Alternatively, add "0" if "." is the start of the string
    string = string.replace(" .", " 0.")
    string = string.replace("{.", "{0.")
    # if empty, return empty string
    if len(string) == 0:
        return string
    if string[0] == ".":
        string = "0" + string

    # to consider: get rid of e.g. "k = " or "q = " at beginning
    if len(string.split("=")) == 2:
        if len(string.split("=")[0]) <= 2:
            string = string.split("=")[1]

    # fix sqrt3 --> sqrt{3}
    string = _fix_sqrt(string)

    # remove spaces
    string = string.replace(" ", "")

    # \frac1b or \frac12 --> \frac{1}{b} and \frac{1}{2}, etc. Even works with \frac1{72} (but not \frac{72}1). Also does a/b --> \\frac{a}{b}
    string = _fix_fracs(string)

    # manually change 0.5 --> \frac{1}{2}
    if string == "0.5":
        string = "\\frac{1}{2}"

    # NOTE: X/Y changed to \frac{X}{Y} in dataset, but in simple cases fix in case the model output is X/Y
    string = _fix_a_slash_b(string)

    return string

def is_equiv(str1, str2, verbose=False):
    if str1 is None and str2 is None:
        print("WARNING: Both None")
        return True
    if str1 is None or str2 is None:
        return False

    try:
        ss1 = _strip_string(str1)
        ss2 = _strip_string(str2)
        if ss2.isdigit():
            try:
                to_return = is_correct(ss1, ss2)
                return to_return
            except:
                print('not a number')
        if verbose:
            print(ss1, ss2)
        return ss1 == ss2
    except:
        return str1 == str2

In [59]:
rootdir = "test"
answer_list = []
question_list = []
for subdir, dirs, files in os.walk(rootdir):
    print('current directory:', subdir)
    file_num = 0
    for file in files:
        file_num += 1
        if file_num > 50:
            break
        with open(os.path.join(subdir, file), 'r') as fp:
            try:
                problem_data = json.load(fp)
            except Exception as e:
                print(f"Error loading JSON from {file}", e)
                raise e
            prob_level = problem_data["level"]
            #prob_type = problem_data["type"]
            try:
                prob_level = int(prob_level.split("Level ")[1])
            except:
                prob_level = None
            answer = remove_boxed(last_boxed_only_string(problem_data["solution"]))
            answer_list.append(answer)
            question_list.append((problem_data["problem"], file))


current directory: test
current directory: test/counting_and_probability
current directory: test/intermediate_algebra
current directory: test/number_theory
current directory: test/precalculus
current directory: test/prealgebra
current directory: test/geometry
current directory: test/algebra


In [40]:
with open('answers_golden_50.txt', 'w') as f:
    f.write('\n'.join(a for a in answer_list))

In [41]:
with open('answers_golden_50.txt', 'r') as file:
    golden = file.readlines()

In [17]:
with open('answers_gpt3_io.txt', 'r') as file:
    output_io = file.readlines()

with open('answers_gpt3_cot_0Shot.txt', 'r') as file:
    output_cot_0Shot = file.readlines()

# with open('answers_gpt3_cot_2Shot.txt', 'r') as file:
#     output_cot_2Shot = file.readlines()

In [18]:
def evaluation(output, golden):
    correct = 0
    length = len(golden)
    print(length)
    list_wrong = []
    i = 0

    for (o, g) in zip(output, golden):
        #print(o, g)
        try:
            equiv = is_equiv(o, g)
            #print(equiv)
        except:
            equiv = False
        if equiv:
            correct += 1
        else:
            list_wrong.append(i)
        i += 1
    
    return correct, correct/length, list_wrong

In [76]:
c, a, wrong = evaluation(output_io, golden)
c, a

98


(10, 0.10204081632653061)

In [129]:
c, p, l = evaluation(output_cot_0Shot, golden)
c, p

98


(20, 0.20408163265306123)

In [19]:
def Answer_Agent_debate(ori_question='', rounds=1, num_agents=3):
    message_user_prior = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your answer in the format: ###your answer###. \
    Give out the answer in numerical format. \
    Question: {ori_question} \
    Let's think step by step. \
    Answer: \
    """

    message = \
    f"""Given a question, and some previous reasoning, recheck the answers and give out your explanation. \
    If you think your previous answer is wrong, provide the correct answer and your reasoning for it. \
    If you think your previous answer is correct, explain why it is correct. \
    Make sure to include your final answer in the format: ###answer###. \
    Question: {ori_question} \
    """

    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer mathematical questions accurately."}]
    
    all_responses = {}
    for i in range(num_agents):
        all_responses[i] = {'user': [], 'assistant': []}
    
    response_list = Answer_Agent_CoT_0Shot(ori_question, num_agents)
    for j in range(len(response_list)):
        all_responses[j]['user'].append(message_user_prior)
        all_responses[j]['assistant'].append(response_list[j])
    
    for r in range(rounds):
        for i in range(num_agents):
            new_messages = messages
            #special case for corners r-1
            new_messages.append({"role": "user", "content": all_responses[i]['user'][0]})
            new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][0]})
            
            if r != 0:
                new_messages.append({"role": "user", "content": all_responses[i]['user'][r]})
                new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][r]})
            
            other_responses = response_list[:i] + response_list[i+1:]
            message += f'Reasoning from other agents: '
            for k in range(len(other_responses)):
                message += f'Agent {k+1}: {other_responses[k]}'
            message += 'Thought:'
            new_messages.append({"role": "user", "content": message})
            
            response_message = client.chat.completions.create(model=model, messages=new_messages, temperature=temperature)
            response = response_message.choices[0].message.content
            all_responses[i]['user'].append(message)
            all_responses[i]['assistant'].append(response)

    return all_responses

In [ ]:
result = []
for li in l:
    result.append(Answer_Agent_debate(split[li], 2, 3))
    #too much input to model

In [ ]:
result

In [1]:
central pool
whether 3 agents agree with the pool

SyntaxError: invalid syntax (663562552.py, line 1)

In [20]:
def Answer_Agent_debate(ori_question='', rounds=1, num_agents=3):
    message_user_prior = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your answer in the format: ###your answer###. \
    Give out the answer in numerical format. \
    Question: {ori_question} \
    Let's think step by step. \
    Answer: \
    """

    message = \
    f"""Given some potential answers given by other agents: \
    if you think your previous answer is wrong, and one of the potential answers is correct, \
    provide the correct answer and your reasoning for it; \
    if you think your previous answer is correct, explain why it is correct. \
    Make sure to include your final answer in the format: ###answer###. \
    """
    #    Question: {ori_question} \

    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer mathematical questions accurately."}]
    
    all_responses = {}
    for i in range(num_agents):
        all_responses[i] = {'user': [], 'assistant': []}
    response_pool = set()
    
    response_list = Answer_Agent_CoT_0Shot(ori_question, num_agents)
    for j in range(len(response_list)):
        all_responses[j]['user'].append(message_user_prior)
        try:
            response_format = re.search('###(.*)###', response_list[j]).group(1)
            all_responses[j]['assistant'].append(response_format)
            extracted = extract_answer(response_format)
            if extracted != INVALID_ANS:
                response_pool.add(extracted)
            else:
                response_pool.add(response_format)
        except:
            response_format = response_list[j]
            all_responses[j]['assistant'].append('no answer')
    
    for r in range(rounds):
        for i in range(num_agents):
            new_messages = messages
            new_message = message

            new_messages.append({"role": "user", "content": all_responses[i]['user'][0]})
            new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][0]})
            
            if r != 0:
                new_messages.append({"role": "user", "content": all_responses[i]['user'][r]})
                new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][r]})

            potential_answers = list(response_pool)
            new_message += f'Potential answers: '
            num_ans = 1
            for pa in potential_answers:
                new_message += f'{num_ans}:{pa};'
                num_ans += 1
            new_messages.append({"role": "user", "content": new_message})
            
            # other_responses = response_list[:i] + response_list[i+1:]
            # message += f'Reasoning from other agents: '
            # for k in range(len(other_responses)):
            #     message += f'Agent {k+1}: {other_responses[k]}'
            # message += 'Thought:'
            # new_messages.append({"role": "user", "content": message})
            
            response_message = client.chat.completions.create(model=model, messages=new_messages, temperature=temperature)
            response = response_message.choices[0].message.content
            all_responses[i]['user'].append(new_message)
            all_responses[i]['assistant'].append(response)

    #put all answers in a pool, and then do majority vote
    all_responses_pool = []
    for i in range(num_agents):
        answer_i = all_responses[i]['assistant'][-1]
        all_responses_pool.append(answer_i)
        # try:
        #     response_format = re.search('###(.*)###', answer_i).group(1)
        #     extracted = extract_answer(response_format)
        #     if extracted != INVALID_ANS:
        #         all_responses_pool.append(extracted)
        # except:
        #     continue

    #return all_responses, max(set(all_responses_pool), key=all_responses_pool.count)
    return all_responses, all_responses_pool

In [22]:
rootdir = "test"

In [ ]:
for subdir, dirs, files in os.walk(rootdir):
    print('current directory:', subdir)
    file_num = 0
    for file in files:
        file_num += 1
        if file_num > 1:
            break
        with open(os.path.join(subdir, file), 'r') as fp:
            try:
                problem_data = json.load(fp)
            except Exception as e:
                print(f"Error loading JSON from {file}", e)
                raise e
            prob_level = problem_data["level"]
            #prob_type = problem_data["type"]
            try:
                prob_level = int(prob_level.split("Level ")[1])
            except:
                prob_level = None
            response, ans = Answer_Agent_debate(problem_data['problem'], 2, 3)
            print(response, ans)

In [31]:
all_answer_gpt3_text_cot_0s_agent3 = []
ori_reasoning_cot_0s = []
all_counts = -1
for subdir, dirs, files in os.walk(rootdir):
    print('current directory:', subdir)
    file_num = 0
    for file in files:
        file_num += 1
        if file_num > 50:
            break
        with open(os.path.join(subdir, file), 'r') as fp:
            try:
                problem_data = json.load(fp)
            except Exception as e:
                print(f"Error loading JSON from {file}", e)
                raise e
            prob_level = problem_data["level"]
            #prob_type = problem_data["type"]
            try:
                prob_level = int(prob_level.split("Level ")[1])
            except:
                prob_level = None
            response, ans = Answer_Agent_debate(problem_data['problem'], 2, 3)
            all_answer_gpt3_text_cot_0s_agent3.append(ans)
            ori_reasoning_cot_0s.append(response)

current directory: test
current directory: test/counting_and_probability
current directory: test/intermediate_algebra
current directory: test/number_theory
current directory: test/precalculus
current directory: test/prealgebra
current directory: test/geometry
current directory: test/algebra


In [32]:
new_ans = []
for aa in all_answer_gpt3_text_cot_0s_agent3:
    try:
        ans = re.search('###(.*)###', aa[0]).group(1)
        ans = _strip_string(ans)
        if ans.isdigit():
            ans = extract_answer(ans)
        new_ans.append(ans)
    except:
        try:
            ans = re.search('###(.*)###', aa[1]).group(1)
            ans = _strip_string(ans)
            if ans.isdigit():
                ans = extract_answer(ans)
            new_ans.append(ans)
        except:
            try:
                ans = re.search('###(.*)###', aa[2]).group(1)
                ans = _strip_string(ans)
                if ans.isdigit():
                    ans = extract_answer(ans)
                new_ans.append(ans)
            except:
                new_ans.append('')


In [42]:
c, p, l = evaluation(new_ans, golden)
c, p

350


(106, 0.3028571428571429)

In [43]:
with open('answers_gpt3_cot_0Shot_3ensemble_50.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in new_ans))

In [107]:
gl = []
for i in range(7):
    c, p, l = evaluation(new_ans[i*50:i*50+50], golden[i*50:i*50+50])
    if i == 3:
        gl = l
    print(c, p)

50
11 0.22
50
7 0.14
50
18 0.36
50
6 0.12
50
29 0.58
50
12 0.24
50
23 0.46


In [108]:
gl

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 29,
 30,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 45,
 46,
 47,
 48,
 49]

In [147]:
i = 170

In [148]:
question_list[i]

('Find the matrix $\\mathbf{M}$ that swaps the rows of a matrix.  In other words,\n\\[\\mathbf{M} \\begin{pmatrix} a & b \\\\ c & d \\end{pmatrix} = \\begin{pmatrix} c & d \\\\ a & b \\end{pmatrix}.\\]If no such matrix $\\mathbf{M}$ exists, then enter the zero matrix.',
 '945.json')

In [149]:
new_ans[i], golden[i]

('\\begin{pmatrix}0&1\\1&0\\end{pmatrix}',
 '\\begin{pmatrix} 0 & 1 \\\\ 1 & 0 \\end{pmatrix}\n')

In [150]:
all_answer_gpt3_text_cot_0s_agent3[i]

['The correct answer is: $\\begin{pmatrix} 0 & 1 \\\\ 1 & 0 \\end{pmatrix}$. This matrix correctly swaps the rows of a 2x2 matrix. The potential answer $\\begin{pmatrix} a & d \\\\ c & b \\end{pmatrix}$ does not swap the rows, so it is not the correct matrix for this specific task. ###$\\begin{pmatrix} 0 & 1 \\\\ 1 & 0 \\end{pmatrix}$###',
 'The correct answer among the potential answers is: 2: The final answer is: $\\begin{pmatrix} a & d \\\\ c & b \\end{pmatrix}$.\n\nTo see why this is the correct answer, we can verify the matrix multiplication:\n\n$\\begin{pmatrix} a & d \\\\ c & b \\end{pmatrix} \\begin{pmatrix} a & b \\\\ c & d \\end{pmatrix} = \\begin{pmatrix} a^2 + cd & ab + bd \\\\ ac + cd & bc + bd \\end{pmatrix} = \\begin{pmatrix} c & d \\\\ a & b \\end{pmatrix}$\n\nThe resulting matrix is indeed $\\begin{pmatrix} c & d \\\\ a & b \\end{pmatrix}$, which swaps the rows of the original matrix. Therefore, $\\begin{pmatrix} a & d \\\\ c & b \\end{pmatrix}$ is the correct matrix t

In [119]:
re.search('###(.*)###', all_answer_gpt3_text_cot_0s_agent3[6][0]).group(1)

'$\\frac{30}{100}$'